# Segmenting And Clustering Neighborhoods in Toronto
# Part III

In [1]:
!pip install geopy

In [2]:
#Import Libraries
import pandas as pd
import bs4
import requests
from urllib.request import urlopen as uReq
import folium # map rendering library
from geopy.geocoders import Nominatim
import wikipedia as wiki

In [28]:
#Get the html source
my_html = wiki.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(my_html)[0]
df.rename({"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"}, axis=1, inplace=True)
df = df[df.Borough != 'Not assigned']
df.loc[df['Neighborhood'] == 'Not assigned', ['Neighborhood']] = df['Borough']
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head(20)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [29]:
df_Geodata = pd.read_csv("https://cocl.us/Geospatial_data") 
df_Geodata.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_Geodata.head(10)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [31]:
#merge the 2 dataframe into one
df_merge = pd.merge(df, df_Geodata, on='PostalCode')

In [32]:
#Borough that contains Toronto
dfToronto = df_merge[df_merge['Borough'].str.contains('Toronto')]
dfToronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [33]:
## checking unique borough names in Toronto
new_df = dfToronto
new_df['Borough'].unique()

array(['East Toronto', 'Central Toronto', 'Downtown Toronto',
       'West Toronto'], dtype=object)

In [34]:
#new_df.loc[new_df['Borough'].isin(['Central Toronto','Downtown Toronto','East Toronto','West Toronto'])]
Toronto_data = new_df[new_df['Borough'].str.contains("Toronto")]
print(Toronto_data['Borough'].unique())
print('These 4 boroughs contain total', Toronto_data.shape[0], 'postal codes.')

['East Toronto' 'Central Toronto' 'Downtown Toronto' 'West Toronto']
These 4 boroughs contain total 38 postal codes.


In [35]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="trnt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
LIMIT = 100
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [36]:
# create map of Scarborough, Toronto using latitude and longitude values
TorontoMap= folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(dfToronto['Latitude'], dfToronto['Longitude'], dfToronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(TorontoMap) 
TorontoMap

# Getting info from 4Square

In [37]:
CLIENT_ID = '3ROJCKT1XU5BMTM5GPTNAMPBRGX5D1FV0AHVTECP435W0E1W' # your Foursquare ID
CLIENT_SECRET = 'EZG5BIHT4MHTNRCKMS1GHNBW5OJ52VSFLJ2D0OZ1VWEV1F1W' 
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3ROJCKT1XU5BMTM5GPTNAMPBRGX5D1FV0AHVTECP435W0E1W
CLIENT_SECRET:EZG5BIHT4MHTNRCKMS1GHNBW5OJ52VSFLJ2D0OZ1VWEV1F1W


In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
TorontoVenues = getNearbyVenues(names=dfToronto['PostalCode'],
                                   latitudes=dfToronto['Latitude'],
                                   longitudes=dfToronto['Longitude']
                                  )
print('The total number of venues returned is ', Toronto_venues.shape[0])
TorontoVenues.head(10)

The total number of venues returned is  1701


,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
5,M4K,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
6,M4K,43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
7,M4K,43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
8,M4K,43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
9,M4K,43.679557,-79.352188,Messini Authentic Gyros,43.677827,-79.350569,Greek Restaurant


In [40]:
print('Total Toronto PostalCodes:', dfToronto['PostalCode'].shape[0])
print('Total Toronto PostalCode with venues:', TorontoVenues['PostalCode'].unique().shape[0])

Total Toronto PostalCodes: 38
Total Toronto PostalCode with venues: 38


In [41]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add PostalCode column back to dataframe
Toronto_onehot['PostalCode'] = Toronto_venues['PostalCode'] 

# move PostalCode column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]
Toronto_onehot.head(10)

,PostalCode,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,M4K,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,M4K,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,M4K,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,M4K,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,M4K,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
# Grouping
PostalGroup=Toronto_onehot.groupby('PostalCode').mean().reset_index()
PostalGroup

,PostalCode,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.20000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4K,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.023810,...,0.000000,0.02381,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810
2,M4L,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M4M,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.052632,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316
4,M4N,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M4P,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4R,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500
7,M4S,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.026316,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M4T,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4V,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.071429,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000


# Getting Venues in the Hood

In [43]:
def CommonVenues(row, TotalVenues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:TotalVenues]

In [44]:
import numpy as np

# Analysis
TotalVenues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(TotalVenues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
PostalCodeVenueSorted = pd.DataFrame(columns=columns)
PostalCodeVenueSorted['PostalCode'] = PostalGroup['PostalCode']

for ind in np.arange(PostalGroup.shape[0]):
    PostalCodeVenueSorted.iloc[ind, 1:] = CommonVenues(PostalGroup.iloc[ind, :], TotalVenues)

PostalCodeVenueSorted

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,M4E,Neighborhood,Health Food Store,Other Great Outdoors,Trail,Pub,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,...,Dumpling Restaurant,Yoga Studio,Diner,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Electronics Store,Department Store,Dessert Shop
1,M4K,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,...,Restaurant,Caribbean Restaurant,Pub,Diner,Dessert Shop,Spa,Liquor Store,Juice Bar,Fruit & Vegetable Store,Indian Restaurant
2,M4L,Park,Pizza Place,Italian Restaurant,Pet Store,Gym,Pub,Movie Theater,Sandwich Place,Burrito Place,...,Brewery,Liquor Store,Fish & Chips Shop,Steakhouse,Sushi Restaurant,Fast Food Restaurant,Ice Cream Shop,Fruit & Vegetable Store,Deli / Bodega,German Restaurant
3,M4M,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Gym / Fitness Center,Brewery,...,Sandwich Place,Cheese Shop,Chinese Restaurant,Clothing Store,Park,Convenience Store,Stationery Store,Coworking Space,Neighborhood,Music Store
4,M4N,Park,Swim School,Bus Line,Yoga Studio,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,...,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Discount Store,Fish Market,Diner,Dessert Shop
5,M4P,Playground,Breakfast Spot,Sandwich Place,Gym,Grocery Store,Hotel,Park,Food & Drink Shop,Clothing Store,...,Doner Restaurant,Donut Shop,Dumpling Restaurant,Ethiopian Restaurant,Electronics Store,Discount Store,Event Space,Falafel Restaurant,Dog Run,Department Store
6,M4R,Coffee Shop,Yoga Studio,Chinese Restaurant,Spa,Sporting Goods Shop,Mexican Restaurant,Salon / Barbershop,Bagel Shop,Diner,...,Rental Car Location,Fast Food Restaurant,Clothing Store,Gym / Fitness Center,Park,Donut Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Cosmetics Shop
7,M4S,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Thai Restaurant,Restaurant,Café,Coffee Shop,Italian Restaurant,...,Diner,Fried Chicken Joint,Costume Shop,Discount Store,Gym,Seafood Restaurant,Brewery,Flower Shop,Pharmacy,Farmers Market
8,M4T,Playground,Tennis Court,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,...,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Diner,Fish Market,Dessert Shop,Department Store,Deli / Bodega
9,M4V,Pub,Coffee Shop,Fried Chicken Joint,Bagel Shop,Vietnamese Restaurant,Liquor Store,Pizza Place,Supermarket,Light Rail Station,...,Sushi Restaurant,Sports Bar,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Event Space


# K-Means Clustering

In [45]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 10

Toronto_group_clustering = PostalGroup.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_group_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([8, 0, 0, 0, 2, 7, 0, 0, 3, 0])

In [46]:
# add clustering labels
PostalCodeVenueSorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge Toronto_group with Toronto_data to add latitude/longitude for each neighborhood aka, Postcode
Toronto_merged = Toronto_merged.join(PostalCodeVenueSorted.set_index('PostalCode'), on='PostalCode')
Toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,8,Neighborhood,Health Food Store,Other Great Outdoors,Trail,...,Dumpling Restaurant,Yoga Studio,Diner,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Electronics Store,Department Store,Dessert Shop
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,...,Restaurant,Caribbean Restaurant,Pub,Diner,Dessert Shop,Spa,Liquor Store,Juice Bar,Fruit & Vegetable Store,Indian Restaurant
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Pizza Place,Italian Restaurant,Pet Store,...,Brewery,Liquor Store,Fish & Chips Shop,Steakhouse,Sushi Restaurant,Fast Food Restaurant,Ice Cream Shop,Fruit & Vegetable Store,Deli / Bodega,German Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Italian Restaurant,...,Sandwich Place,Cheese Shop,Chinese Restaurant,Clothing Store,Park,Convenience Store,Stationery Store,Coworking Space,Neighborhood,Music Store
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Swim School,Bus Line,Yoga Studio,...,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Discount Store,Fish Market,Diner,Dessert Shop
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,7,Playground,Breakfast Spot,Sandwich Place,Gym,...,Doner Restaurant,Donut Shop,Dumpling Restaurant,Ethiopian Restaurant,Electronics Store,Discount Store,Event Space,Falafel Restaurant,Dog Run,Department Store
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Coffee Shop,Yoga Studio,Chinese Restaurant,Spa,...,Rental Car Location,Fast Food Restaurant,Clothing Store,Gym / Fitness Center,Park,Donut Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Cosmetics Shop
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,...,Diner,Fried Chicken Joint,Costume Shop,Discount Store,Gym,Seafood Restaurant,Brewery,Flower Shop,Pharmacy,Farmers Market
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,3,Playground,Tennis Court,Discount Store,Filipino Restaurant,...,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Diner,Fish Market,Dessert Shop,Department Store,Deli / Bodega
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,0,Pub,Coffee Shop,Fried Chicken Joint,Bagel Shop,...,Sushi Restaurant,Sports Bar,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Event Space


# Visualizing Clusters in Map

In [47]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.coolwarm(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['PostalCode'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters